# Basic Numerical Methods for Option Pricing Approximation

## Implicit Finite-Difference Method

#### We assume a call option as the following:
$$Call = f(S_t, t)$$

#### Expand it with Taylor series:
$$\Delta f = \frac{\partial f}{\partial S_t}\Delta S_t + \frac{\partial f}{\partial t}\Delta t + \frac{\partial^2 f}{2\partial S_t^2}\Delta S^2_t + \frac{\partial^2 f}{2\partial t^2}\Delta t^2 +
\frac{\partial^2 f}{2\partial S_t \partial t}\Delta S_t \Delta t + ...$$


#### Note that the movement of the stock price on which the option depends follows a generalized Wiener process:
$$\Delta S_t = \mu S_t \Delta t + \sigma S_t \Delta z$$
$$\Delta z = \epsilon \sqrt{\Delta t},  \, \, \, \epsilon ~ N(0, 1)$$

#### Taking out higher terms of derivatives $\Delta t$, we get:
$$\Delta f = \frac{\partial f}{\partial S_t} * (\mu S_t \Delta t + \sigma S_t \Delta z) + \frac{\partial f}{\partial t}\Delta t + \frac{\partial^2 f}{2\partial S_t^2} S^2_t \sigma ^2 \Delta t$$
$$= (\frac{\partial f}{\partial t} + \frac{\partial f}{\partial S_t} * \mu S_t + \frac{\partial^2 f}{2\partial S_t^2} S^2_t \sigma ^2) * \Delta t + \frac{\partial f}{\partial S_t}S_t\sigma \Delta z $$

#### Building Portfolios $\prod$ by shorting one derivative and long an amount of $\frac{\partial f}{\partial S_t}$ of shares:
$$\prod = -1 * f + (\frac{\partial f}{\partial S_t}) * S_t; \,\,\,\,\,\, \Delta \prod = -\Delta f + (\frac{\partial f}{\partial S_t}) \Delta S_t$$
#### which equals to,
$$\Delta \prod = -(\frac{\partial f}{\partial t} + \frac{\partial f}{\partial S_t} * \mu S_t + \frac{\partial^2 f}{2\partial S_t^2} S^2_t \sigma ^2) * \Delta t - \frac{\partial f}{\partial S_t}S_t\sigma \Delta z + (\frac{\partial f}{\partial S_t})(\mu S_t \Delta t + \sigma S_t \Delta z)$$

#### The elimination of the volatility terms $\Delta z$ corresponds to the riskless nature of the portfolio we built:
$$-\frac{\partial f}{\partial S_t}S_t \sigma \Delta z + (\frac{\partial f}{\partial S_t}) S_t \sigma \Delta z = 0$$

#### Therefore, we can simplify the expression of $\Delta \prod$ as:
$$\Delta \prod = -(\frac{\partial f}{\partial t} + \frac{\partial f}{\partial S_t} * \mu S_t + \frac{\partial^2 f}{2\partial S_t^2} S^2_t \sigma ^2 - \frac{\partial f}{\partial S_t}\mu S_t) \Delta t$$
$$=(-\frac{\partial f}{\partial t} - \frac{\partial^2 f}{2\partial S_t^2} S^2_t \sigma ^2 )\Delta t$$


#### Considering the risklessness of the portfolio, it then follows that $\Delta \prod = r \prod \Delta t$:
$$\frac{\partial f}{\partial t} + \frac{\partial^2 f}{2\partial S_t^2} S^2_t \sigma ^2 = r(f - \frac{\partial f}{\partial S_t} * S_t)$$

#### Therefore, we arrived at our desired equation:
$$\frac{\partial f}{\partial t} + (r-q)S_t\frac{\partial f}{\partial S_t} + \frac{1}{2}S^2_t \sigma ^2\frac{\partial^2 f}{\partial S_t^2}  = rf$$
#### where q is the dividend yield paid by the stock.

#### Now we employ the finite difference method to solve this partial differential equation

#### From book $Options, Futures, and \, Other \, Deriatives$ by John C. Hull, we have the implicit form, in non-dividend cases:
$$\frac{f_{i+1, j} - f_{i,j}}{\Delta t} + rj \Delta S_t \frac{f_{i, j+1} - f_{i, j-1}}{2\Delta S_t} + \frac{1}{2} (\sigma j)^2 \Delta S_t^2 \frac{f_{i, j+1} + f_{i, j-1} - 2f_{i, j}}{\Delta S_t^2} = rf_{i, j}$$

#### Simplify the expresison by forward approximation:
$$f_{i+1, j} = a_j f_{i,j-1} + b_j f_{i, j} + c_j f_{i, j+1}$$
#### where
$$a_j =  \frac{1}{2} \Delta t (r*j - \sigma ^2 j^2) $$
$$b_j = 1 + \Delta t(\sigma ^2 j^2 + r) $$
$$c_j = -\frac{1}{2}\Delta t (r*j + \sigma ^2 j^2)$$

#### Thus, we have the forward approximation in matrix form as:
$$F_{i+1} = BF_{i}$$
$$F_{i} = B^{-1}F_{i+1}$$
#### where
$$ F_i = \begin{bmatrix}
f_{i,1} \\
f_{i,2} \\
... \\
... \\
f_{i,M-1} 
\end{bmatrix}  $$
$$ B = \begin{bmatrix}
b_1 & c_1 & 0 & ... & ... \\
a_2 & b_2 & c_2 & ... & ... \\
0 & a_3 & b_3 & c_3 & ... \\
... & ... & ... & b_{M-2} & c_{M-2} \\
0 & 0 & ... & a_{M-1} & b_{M-1} 
\end{bmatrix}  $$

#### We shall first construct the grid in matrix form

In [97]:
import numpy as np
import matplotlib.pyplot as plt
from random import gauss
from tqdm import tqdm

In [86]:
M = 5000 # Grid number for finite difference
K = 230 # Strike price
S = 276.1 # Assume the current price to be S_0
S_max = 500 # The level S_max is chosen so that one of these is the current stock price.
T = 58 / 365 # Time to maturity of the option
r = 0.16 / 100 # The continuously compounded risk-free rate
q = 0 # Assume a non-dividend case
delta_s = S_max / M #Defining the subinterval on the axes of price [0, S_max]
delta_t = T / M #Defining the subinterval on the axes of time [0, T]
sig = 0.407530933 # Stock price volatility

In [87]:
def get_call(M, K, delta_s):
    call_matrix = np.matrix(np.array([0.0]*(M+1)*(M+1)).reshape((M+1, M+1)))
    call_matrix[:, 0] = 0
    call_matrix[:, M] = float(S_max - K)
    for i in range(M+1):
        call_matrix[M, i] = float(max(delta_s * i - K, 0))
    return call_matrix

def get_put(M, K, delta_s):
    put_matrix = np.matrix(np.array([0.0]*(M+1)*(M+1)).reshape((M+1, M+1)))
    put_matrix[:, 0] = float(K)
    put_matrix[:, M] = 0
    for i in range(M+1):
        put_matrix[M, i] = float(max(K - delta_s * i, 0))
    return put_matrix

In [88]:
get_call(M, K, delta_s)

matrix([[  0. ,   0. ,   0. , ...,   0. ,   0. , 270. ],
        [  0. ,   0. ,   0. , ...,   0. ,   0. , 270. ],
        [  0. ,   0. ,   0. , ...,   0. ,   0. , 270. ],
        ...,
        [  0. ,   0. ,   0. , ...,   0. ,   0. , 270. ],
        [  0. ,   0. ,   0. , ...,   0. ,   0. , 270. ],
        [  0. ,   0. ,   0. , ..., 269.8, 269.9, 270. ]])

In [89]:
get_put(M, K, delta_s)

matrix([[230. ,   0. ,   0. , ...,   0. ,   0. ,   0. ],
        [230. ,   0. ,   0. , ...,   0. ,   0. ,   0. ],
        [230. ,   0. ,   0. , ...,   0. ,   0. ,   0. ],
        ...,
        [230. ,   0. ,   0. , ...,   0. ,   0. ,   0. ],
        [230. ,   0. ,   0. , ...,   0. ,   0. ,   0. ],
        [230. , 229.9, 229.8, ...,   0. ,   0. ,   0. ]])

#### To solve the matrix equation, we shall first calculate the coefficient matrix B

In [90]:
def get_diff(j, sig=0.4):
    sig_j_square = (sig * j) **2 #Here we have (sigma^2 * j^2)
    a = 0.5 * delta_t * ( (r-q) * j - sig_j_square)
    b = 1 + delta_t * (sig_j_square + r)
    c = -0.5 * delta_t * ( (r-q) * j + sig_j_square)
    return a, b, c


def get_coeff(M):
    matrix = np.matrix(np.array([0.0]*(M-1)*(M-1)).reshape((M-1, M-1)))
    a1, b1, c1 = get_diff(1)
    am_1, bm_1, cm_1 = get_diff(M - 1)
    
    matrix[0,0] = b1
    matrix[0,1] = c1
    
    matrix[M-2, M-3] = am_1
    matrix[M-2, M-2] = bm_1
    
    for i in range(2, M-1):
        a, b, c = get_diff(i)
        matrix[i-1, i-2] = a
        matrix[i-1, i-1] = b
        matrix[i-1, i] = c    
    return matrix

In [100]:
def AM(signal, M):
    if signal == "call":
        f_matrix = get_call(M, K, delta_s)
    elif signal == "put":
        f_matrix = get_put(M, K, delta_s)
    
    coeff = get_coeff(M)
    inverse_coeff = np.linalg.inv(coeff)

    for i in range(M, 0, -1):  # Starting from the bottom row of the matrix
        Fi_1 = f_matrix[i, 1:M]
        Fi = inverse_coeff * Fi_1.reshape((M-1, 1))  # take the product of inverse_coeff and F_{i+1} to get F_i
        Fi = list(np.array(Fi.reshape(1, M-1))[0]) 
        for j in range(0, M-1):
            if signal == "call":
                Fi[j] = np.maximum(Fi[j], ((j+1) * delta_s - K))  # early exercise condition for call option
            elif signal == "put":
                Fi[j] = np.maximum(Fi[j], (K - ((j+1) * delta_s)))  # early exercise condition for put option
        
        f_matrix[i-1, 1:M] = Fi  # Load the row F_i through calculation to the top of F_{i+1}
    
    i = int(np.round(S / delta_s, 0))  # Return the estimated option price corresponding to the current stock price
    return f_matrix[0, i]


def EUR(signal, M):
    if signal == "call":
        f_matrix = get_call(M, K, delta_s)
    elif signal == "put":
        f_matrix = get_put(M, K, delta_s)
    
    coeff = get_coeff(M)
    inverse_coeff = np.linalg.inv(coeff)

    for i in range(M, 0, -1):  # Starting from the buttom row of the matrix
            Fi_1 = f_matrix[i, 1:M]
            Fi = inverse_coeff * Fi_1.reshape((M-1, 1))  # take the product of inverse_coeff and F_{i+1} to get F_i
            Fi = list(np.array(Fi.reshape(1, M-1))[0]) 
            f_matrix[i-1, 1:M] = Fi
    i = np.round(S/delta_s, 0)  # Return the estimated option price corresponding to the current stock price
    
    return f_matrix[0, int(i)]

In [99]:
eur_put = EUR("put", M)
eur_call = EUR("call", M)
am_put = AM("put", M)
am_call = AM("call", M)
print(f'Approximated prices for call and put European options are {eur_call}, {eur_put}.')
print(f'Approximated prices for call and put American options are {am_call}, {am_put}.')

100%|██████████████████████████████████████| 5000/5000 [00:49<00:00, 102.01it/s]

Approximated prices for call and put American options are 48.661930448236795, 2.503705910937936.


## Crank-Nicolson Method

#### Crank-Nicolson method is more stable than the explicit method and quickly converges to the solution.
#### Derivative terms thus become
$$\frac{\partial f}{\partial S_t} = \frac{f_{i, j+1} - f_{i, j-1}}{2 \Delta S_t} \Rightarrow \frac{\partial f}{\partial S_t} \approx \frac{1}{2} \left( \frac{f_{i+1, j+1} - f_{i+1, j-1}}{2\Delta S_t} + \frac{f_{i, j+1} - f_{i, j-1}}{2\Delta S_t}\right)$$
$$\frac{\partial^2 f}{\partial S_t^2} = \frac{f_{i, j+1} - 2f_{i, j} + f_{i, j-1}}{\Delta S_t ^2} \Rightarrow \frac{\partial^2 f}{\partial S_t^2} \approx \frac{1}{2} \left( \frac{f_{i+1, j+1} + f_{i+1, j-1} - 2f_{i+1, j}}{\Delta S_t^2} + \frac{f_{i, j+1} + f_{i, j-1} - 2f_{i, j}}{\Delta S_t^2}\right)$$

#### The equation is thus simplified to 
$$-\alpha_j f_{i, j-1} + (1-\beta_j)f_{i, j} - \gamma_j f_{i, j+1} = \alpha_j f_{i+1, j-1} + (1+\beta_j)f_{i+1, j} + \gamma_j f_{i+1, j+1}$$
#### where
$$\alpha_j = \frac{\Delta t}{4}(\sigma^2j^2 - rj)$$
$$\beta_j = -\frac{\Delta t}{2}(\sigma^2j^2 + r)$$
$$\gamma_j = \frac{\Delta t}{4}(\sigma^2j^2 + rj)$$

#### The matrix form thus become
$$M_1f_i = M_2f_{i+1} + b$$
#### where $f_i$ and $b$ are vectors of length $M-1$
$$f_i = \left[f_{i, 1}, f_{i, 2}, ..., f_{i, M-1}\right]^T$$
$$b = \left[\alpha_1(f_{i, 0} + f_{i+1, 0}, 0, ..., 0, \gamma_{M-1}(f_{i, M} + f_{i+1, M})\right]^T$$
#### and $M_1$, $M_2$ are matrix of dimensions $(M-1) \times (M-2)$.
$$ M_1 = \begin{bmatrix}
1-\beta_1 & -\gamma_1 & 0 & ... & 0 \\
-\alpha_2 & 1-\beta_2 & -\gamma_2 & ... & 0 \\
0 & -\alpha 3 & ... & ... & ... \\
... & ... & ... & 1-\beta_{M-2} & -\gamma_{M-2} \\
0 & 0 & ... & -\alpha_{M-1} & 1-\beta_{M-1} 
\end{bmatrix}  $$
$$ M_2 = \begin{bmatrix}
1+\beta_1 & \gamma_1 & 0 & ... & 0 \\
\alpha_2 & 1+\beta_2 & \gamma_2 & ... & 0 \\
0 & \alpha 3 & ... & ... & ... \\
... & ... & ... & 1+\beta_{M-2} & \gamma_{M-2} \\
0 & 0 & ... & \alpha_{M-1} & 1+\beta_{M-1} 
\end{bmatrix}  $$
#### Thus, the iteration becomes
$$f_i = M_1^{-1}M_2f_{i+1} + M_1^{-1}b$$

In [94]:
def get_diff_cn(j, sig=0.4):
    sig_j_square = (sig * j) **2 #Here we have (sigma^2 * j^2)
    a = (delta_t / 4) * (sig_j_square - r*j)
    b = (-delta_t / 2) * (sig_j_square + r)
    c = (delta_t / 4) * (sig_j_square + r*j)
    return a, b, c


def get_coeff_1(M):
    matrix = np.matrix(np.array([0.0]*(M-1)*(M-1)).reshape((M-1, M-1)))
    a1, b1, c1 = get_diff_cn(1)
    am_1, bm_1, cm_1 = get_diff_cn(M - 1)
    
    matrix[0,0] = 1 - b1
    matrix[0,1] = -c1

    matrix[M-2, M-3] = -am_1
    matrix[M-2, M-2] = 1-bm_1
    
    for i in range(2, M-1):
        a, b, c = get_diff_cn(i)
        matrix[i-1, i-2] = -a
        matrix[i-1, i-1] = 1-b
        matrix[i-1, i] = -c    
    return matrix

def get_coeff_2(M):
    matrix = np.matrix(np.array([0.0]*(M-1)*(M-1)).reshape((M-1, M-1)))
    a1, b1, c1 = get_diff_cn(1)
    am_1, bm_1, cm_1 = get_diff_cn(M - 1)
    
    matrix[0,0] = 1 + b1
    matrix[0,1] = c1

    matrix[M-2, M-3] = am_1
    matrix[M-2, M-2] = 1+bm_1
    
    for i in range(2, M-1):
        a, b, c = get_diff_cn(i)
        matrix[i-1, i-2] = a
        matrix[i-1, i-1] = 1+b
        matrix[i-1, i] = c    
    return matrix

In [95]:
def AM_explicit(signal, M):
    if signal == "call":
        f_matrix = get_call(M, K, delta_s)
    elif signal == "put":
        f_matrix = get_put(M, K, delta_s)
    
    coeff_m1 = get_coeff_1(M)
    coeff_m2 = get_coeff_2(M)
    inverse_m1 = np.linalg.inv(coeff_m1)

    for i in range(M, 0, -1):  # Starting from the bottom row of the matrix
        b = np.matrix(np.array([0.0]*(M-1)*(1)).reshape((M-1, 1)))
        b[0] = get_diff_cn(1)[0] * (f_matrix[i-1, 0] + f_matrix[i, 0])
        b[-1] = get_diff_cn(M-1)[2] * (f_matrix[i-1, -1] + f_matrix[i, -1])
        Fi_1 = f_matrix[i, 1:M]
        Fi = inverse_m1 * (coeff_m2 * Fi_1.reshape((M-1, 1))) + (inverse_m1 * b)
        Fi = list(np.array(Fi.reshape(1, M-1))[0])
        
        for j in range(0, M-1):
            if signal == "call":
                Fi[j] = np.maximum(Fi[j], ((j+1) * delta_s - K))  # early exercise condition for call option
            elif signal == "put":
                Fi[j] = np.maximum(Fi[j], (K - ((j+1) * delta_s)))  # early exercise condition for put option
        f_matrix[i-1, 1:M] = Fi  # Load the row F_i through calculation to the top of F_{i+1}
    
    i = int(np.round(S / delta_s, 0))  # Return the estimated option price corresponding to the current stock price
    return f_matrix[0, i]


def EUR_explicit(signal, M):
    if signal == "call":
        f_matrix = get_call(M, K, delta_s)
    elif signal == "put":
        f_matrix = get_put(M, K, delta_s)
    
    coeff_m1 = get_coeff_1(M)
    coeff_m2 = get_coeff_2(M)
    inverse_m1 = np.linalg.inv(coeff_m1)
    
    for i in range(M, 0, -1):  # Starting from the buttom row of the matrix
            b = np.matrix(np.array([0.0]*(M-1)*(1)).reshape((M-1, 1)))
            b[0] = get_diff_cn(1)[0] * (f_matrix[i-1, 0] + f_matrix[i, 0])
            b[-1] = get_diff_cn(M-1)[2] * (f_matrix[i-1, -1] + f_matrix[i, -1])
            Fi_1 = f_matrix[i, 1:M]
            Fi = inverse_m1 * (coeff_m2 * Fi_1.reshape((M-1, 1))) + (inverse_m1 * b)            
            Fi = list(np.array(Fi.reshape(1, M-1))[0]) 
            f_matrix[i-1, 1:M] = Fi
    i = np.round(S/delta_s, 0)  # Return the estimated option price corresponding to the current stock price
    
    return f_matrix[0, int(i)]

In [96]:
eur_put = EUR_explicit("put", M)
eur_call = EUR_explicit("call", M)
am_put = AM_explicit("put", M)
am_call = AM_explicit("call", M)
print(f'Approximated prices for call and put European options are {eur_call}, {eur_put}.')
print(f'Approximated prices for call and put American options are {am_call}, {am_put}.')

Approximated prices for call and put European options are 48.66186876591688, 2.5034004195340414.
Approximated prices for call and put American options are 48.66186876591688, 2.5036473764222396.


#### The option price of our interest are given above. Then, comparing the numerical solutions with the ones derived from the closed-form formula:
$$P_{call} = S_0 N(d_1) - Ke^{-rT}N(d_2)$$
$$P_{put} = Ke^{-rT}N(-d_2) - S_0 N(-d_1)$$
#### where
$$d_1 = \frac{ln(S_0 / K) + (r+ \sigma ^2 / 2)T}{\sigma \sqrt{T}}$$
$$d_2 = \frac{ln(S_0 / K) + (r - \sigma^2 / 2)T}{\sigma \sqrt{T}} = d_1 - \sigma \sqrt{T}$$

In [254]:
from scipy.stats import norm
d1 = (np.log(S/K) + (r + 0.5*(sig**2)) * T) / (sig * np.sqrt(T))
d2 = d1 - (sig * np.sqrt(T))
p_call = (S * norm.cdf(d1)) - (K * np.exp(-r*T) * norm.cdf(d2))
p_put = (K * np.exp(-r*T) * norm.cdf(-d2)) - (S * norm.cdf(-d1))
print(f'Prices for call and put options from the closed-form formula are {call}, {put}.')

Prices for call and put options from the closed-form formula are 48.622359389607766, 2.503462928415934.


#### Accordingly, we found the estimated prices derived from two approaches align with each other.

## Monte Carlo Simulation

#### The movement followed by the underlying market variable in a risk-neutral world remains:
$$\Delta S_t = \mu S_t \Delta t + \sigma S_t \Delta z$$
#### Thus we can write the stepwise expression as,
$$S_{t+\Delta t} - S_t = \mu S_t \Delta t + \sigma S_t \epsilon \sqrt{\Delta t}$$

#### Considering the nature of log normality in practice, rewrite the expression of process as,
$$ \Delta \ln{S_t} = \left( \mu - \frac{\sigma ^2}{2}\right) \Delta t + \sigma \Delta z$$
#### with its discretized form,
$$ \ln{S_{t+\Delta t}} - \ln{S_t} = \left( \mu - \frac{\sigma ^2}{2} \right) \Delta t + \sigma \epsilon \sqrt{\Delta t}$$
$$ S_{t+\Delta t} = S_t \exp \left[ \left( \mu - \frac{\sigma ^2}{2} \right) \Delta t + \sigma \epsilon \sqrt{\Delta t} \right]$$
#### Alternatively, the option price at maturity is 
$$ S_{T} = S_t \exp \left[ \left( \mu - \frac{\sigma ^2}{2} \right) (T -t) + \sigma \epsilon \sqrt{T - t} \right]$$

#### In particular, based on the assumption of a risk-neutral world, the expected mean return $\mu$ of the underlying asset equals the risk-free rate $r$.

#### To conduct Monte Carlo Simulations, while keeping $\mu, S_t, \sigma, T$ constant, we aim at sampling the $\epsilon ~ N(0,1)$ to get a series of sampled values $S_t$. 
#### Then, after calculating the expected payoff, either $max(S_t - K, 0)$ or $max(K - S_t, 0)$, and discounting by $exp(-rT)$, the pricing estimation is finished by taking the mean results.

In [164]:
simulation = 100000 # Number of simulations
# Keeping other parameters as usual

In [167]:
def calculate_st(S, sig, r, T):
    return S * np.exp((r - 0.5 * sig ** 2) * T + sig * np.sqrt(T) * gauss(0.0, 1.0))


def payoff(signal, st, K):
    if signal == "call":
        payoff = max(st - K, 0)
    else:
        payoff = max(K - st, 0)
    return payoff


def option_pricing(signal, S, sig, r, T, K, simulation):
    payoffs = []
    discount = np.exp(-r * T)
    for i in range(simulation):
        st = calculate_st(S, sig, r, T)
        payoffs.append(
            payoff(signal, st, K)
        )
    price = discount * np.sum(payoffs) / simulation
    return price

In [168]:
call = option_pricing("call", S, sig, r, T, K, simulation)
put = option_pricing("put", S, sig, r, T, K, simulation)
print(f'Approximated prices for call and put options are {call}, {put}.')

Approximated prices for call and put options are 48.6195085580708, 2.654746555559286.
